# Multiple Linear Regression

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd

## Dataset and business problem description

This dataset contains information on 50 companies, including their expenses, profits, and which state they operate in. The challenge is to determine how each of the factors available contribute to the profit level of each company.

## Intuition

### Assumptions of linear regression

These assumptions need to be checked before you build your linear regression model to ensure that the resulting model is valid.

1. Linearity: the relationship between the dependent and independent variable(s) is linear. If the relationship is not linear then you'll need to use a non-linear model.
2. Homoscedasticity: the variance of the residuals do not depend on the value of the independent variable (i.e. variance remains the same throughout the dataset). If the residuals vary with X, then the model is not optimal since it isn't capturing all the predictive information in the data.
3. Multivariate normality: residuals are normally distributed around the mean value.
4. Independence of errors: observations are independent of one another. This can be tested with the Durbin Watson statistic.
5. Lack of multicollinearity: when independent variable are correlated with each other, there will be issues in interpreting the coefficients of the model. This can be tested with the Variance Inflation Factor method.

### Dummy variables & intuition

The business problem in this lesson is to see if there is any correlation between spending on R&D, administration, or marketing, as well as the state the startup is operating in; on the profit that the startup is generating. How would you go about creating a model to understand the relationship between these variables and profit? We can use a multiple linear regression for this.

Multiple linear regression is a generalisation of simple linear regression to include multiple independent variables: $y = b_0 + b_1x_1 + \dots + b_Nx_N$. 

The equation coefficients have a similar interpretation to in simple linear regression, where:

* $b_0$ is still the intercept; where the dependent variable is when the independent variables is at 0.
* $b_n$ is the gradient of variable $x_n$; this describes the change in the dependent variable with every unit change in of $x_n$, holding all other independent variables constant.


When adding categorical variables to the model, you'll need to turn them into dummy variables - being careful to not fall into the dummy variable trap. i.e. Use $n-1$ dummy variables when there are $n$ categories; the variable you leave out then becomes the 'default' value, and its effect will be included in the intercept.

### Understanding the P-value - statistical significance

The intuition behind hypothesis testing is that you have two alternate universes:

* $H_0$: Your null hypothesis, the 'default' universe; and,
* $H_1$: Your alternative hypothesis, the universe where the thing you're trying to prove is true.

Put simply, the P-value is the probability of the results that you found occurring, given that we're in a universe where the null hypothesis is true. Statistical significance ($\alpha$) is defined as a P-value threshold below which the event that occurred is so unlikely that you reject the null hypothesis. In other words you are $(1-\alpha)$% sure that we don't live in the 'default' universe (although there's a $\alpha$% chance that we do).

### Building a model (step-by-step)

We have a lot of independent variables; we'll need to decide which ones to keep and which to discard. Why would we want to narrow the variable list down?

* Garbage in, garbage out: there's no guarantee that more variables = better model; especially if there turns out to be multicollinearity in your predictors.
* Explainability: you'll want to be able to explain the effect that each variable has on the outputs, this is hard to do when you have a lot of useless variables.

This course presents 5 methods of building models (stepwise regression refers to methods 2-4):

<div class="alert alert-block alert-warning">
    <b>On the validity of these methods</b><br/> 
    Most of the methods this course presents for finding the 'ideal' model in regression are stepwise selection processes. There are <a href="https://towardsdatascience.com/stopping-stepwise-why-stepwise-selection-is-bad-and-what-you-should-use-instead-90818b3f52df">arguments against this method of selection</a> citing the reason that the F-test used to assess the fit of the regression model (which differs from the t-test used to test each individual coefficient) is designed only for use in performing one hypothesis test, not many in sequence. As a result of this violation, the following are true:
    <ul>
        <li>Standard errors are biased toward 0 (i.e. overly optimistic)</li>
        <li>p-values are biased towards 0 (i.e. overly optimistic)</li>
        <li>Parameter estimates biased away from 0</li>
        <li>Models too complex</li>
    </ul><br/>
    Alternatives presented include the use of LASSO regression and Least Angle Regression to arrive at a subset of predictors that are useful and predictive.

Back to the course material...
</div>

#### All-in

Throw all your variables in. When would you do this?

* You have prior knowledge that all available variables are useful predictors; or,
* You're required (legislation/business rules) to put these variables in; or,
* You're preparing for backward elimination.

#### Backward elimination

1. Select a significance level as a threshold with which to keep a variable in the model (generally $\alpha=0.05$).
2. Fit full model with all possible predictors.
3. Look at the predictor with the highest P-value. If this P-value exceeds the threshold go to Step 4. Otherwise you are done.
4. Remove the predictor and re-fit the model. Go back to Step 3.

#### Forward selection

A much more complex procedure.

1. Select a significance level to enter the model (generally $\alpha=0.05$).
2. Fit regression models $y~x_n$ for all $n$ variables. Select the model with the lowest P-value.
3. Keep the model with the variable selected in Step 2, and fit all possible models $y~x_1 + x_{n-1}$. Select the model where $x_n$ has the lowest P-value. If the P-value is smaller than $\alpha$ then continue adding variables in this manner. Otherwise remove this latest variable where P > $\alpha$ and you are left with your final model.

#### Bidirectional elimination

1. Select a significance level to enter the model, and a significance level to stay in the model.
2. Perform the next step of forward selection (new variables must have P-value < significance level to enter).
3. Perform all steps of backward elimination.
4. Add another variable via forward selection and then run all steps of backward elimination again.
5. Once you get to the point where no new variables can exit, and no old variables can enter, you are done.

#### Score comparison

Most resource intensive approach.

1. Select a criterion of goodness-of-fit (for example, Akaike's Information Criterion).
2. Construct all possible models ($2^N-1$ total combinations - 10 variables means 1,023 possible combinations).
3. Select the model with the best criterion.

We'll focus on using the backward elimination process in this course since it's generally the fastest of all of these methods.

## Building the model
### Importing the dataset

Note there is no need to scale the features in regression since the coefficients for each variable will adjust based on the scale of the feature.

In [2]:
dataset = pd.read_csv('./data/50_Startups.csv')

X = dataset.iloc[:, :-1].values
y = dataset.iloc[:, -1].values

dataset.head()

,R&D Spend,Administration,Marketing Spend,State,Profit
0,165349.20,136897.80,471784.10,New York,192261.83
1,162597.70,151377.59,443898.53,California,191792.06
2,153441.51,101145.55,407934.54,Florida,191050.39
3,144372.41,118671.85,383199.62,New York,182901.99
4,142107.34,91391.77,366168.42,Florida,166187.94


In [3]:
# Checking for NAs
dataset.isna().sum()

R&D Spend          0
Administration     0
Marketing Spend    0
State              0
Profit             0
dtype: int64

### Encoding categorical data

In [4]:
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder

ct = ColumnTransformer(transformers=[(
    'encoder', 
    OneHotEncoder(
# Note the options below are redundant, since the Multiple Linear regression class will handle this automatically
#         # Use this to make sure you know which var is dropped (dataset.State.unique().tolist())[0]
#         categories = [dataset.State.unique().tolist()], 
#         # Use drop = 'first' to avoid dummy variable trap
#         drop = 'first'
    ), 
    [3])
], remainder='passthrough')

X = np.array(ct.fit_transform(X))

X[1:10,]

array([[1.0, 0.0, 0.0, 162597.7, 151377.59, 443898.53],
       [0.0, 1.0, 0.0, 153441.51, 101145.55, 407934.54],
       [0.0, 0.0, 1.0, 144372.41, 118671.85, 383199.62],
       [0.0, 1.0, 0.0, 142107.34, 91391.77, 366168.42],
       [0.0, 0.0, 1.0, 131876.9, 99814.71, 362861.36],
       [1.0, 0.0, 0.0, 134615.46, 147198.87, 127716.82],
       [0.0, 1.0, 0.0, 130298.13, 145530.06, 323876.68],
       [0.0, 0.0, 1.0, 120542.52, 148718.95, 311613.29],
       [1.0, 0.0, 0.0, 123334.88, 108679.17, 304981.62]], dtype=object)

### Splitting the dataset into the Training set and Test set

In [5]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 0)

### Training the Multiple Linear Regression model on the Training set

As noted earlier, this class handles the dummy variable trap automatically for you. It also performs feature selection for you. (See text below these code blocks).

In [6]:
from sklearn.linear_model import LinearRegression
regressor = LinearRegression()
regressor.fit(X_train, y_train)

LinearRegression()

In [7]:
regressor.coef_

array([ 8.66383692e+01, -8.72645791e+02,  7.86007422e+02,  7.73467193e-01,
        3.28845975e-02,  3.66100259e-02])

There are 6 coefficients above (`.coef_` does not include the intercept), which implies that the model has used all variables provided, including all levels of the dummy variable. This runs counter to what was described in the course, that this class would take care of these issues automatically.

When asked this question, teaching assistants noted that you don't need to always drop a level; [citing this link](https://datascience.stackexchange.com/questions/27957/why-do-we-need-to-discard-one-dummy-variable). No comment is made on the automatic variable selection claim.

We'll continue with this model as this is what the course uses, but it would be wise to consider the dummy variable trap in the context of future problems.

### Predicting the Test set results

In [9]:
# Cap printing at 2 dp
np.set_printoptions(precision = 2)

y_pred = regressor.predict(X_test)
y_pred

array([103015.2 , 132582.28, 132447.74,  71976.1 , 178537.48, 116161.24,
        67851.69,  98791.73, 113969.44, 167921.07])

In [10]:
# Concatenate predicted values against actuals
np.concatenate(
    (
        y_pred.reshape(len(y_pred), 1), # Transposing array
        y_test.reshape(len(y_test), 1)
    ),
    axis = 1 # Column-bind (0 is row-bind)
)

array([[103015.2 , 103282.38],
       [132582.28, 144259.4 ],
       [132447.74, 146121.95],
       [ 71976.1 ,  77798.83],
       [178537.48, 191050.39],
       [116161.24, 105008.31],
       [ 67851.69,  81229.06],
       [ 98791.73,  97483.56],
       [113969.44, 110352.25],
       [167921.07, 166187.94]])